#### Imports

In [88]:
import sys
import os

import geopandas as gpd
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

#### Constants

In [89]:
# General
#BASE_PATH = os.getcwd()
BASE_PATH = os.path.dirname(os.getcwd())

# Mapping
CHANGE_TYPE_MAP = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}
CHANGE_STATUS_MAP = {'Land Cleared': 1, 'Materials Dumped': 3, 'Construction Midway': 6, 'Prior Construction': 3, 'Greenland': 0, 'Construction Done': 8, 'Construction Started': 5, 'Materials Introduced': 4, 'Operational': 10,  None: None, 'Excavation': 1}

# Data
COLORS = ['red', 'green', 'blue']
METRICS = ['std', 'mean']

# Columns groups
COLUMNS_TO_DROP = ['index']
DATE_COLUMNS = ['date0', 'date1', 'date2', 'date3', 'date4']
CHANGE_STATUS_COLUMNS = ['change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3', 'change_status_date4']

#### Reading data

In [90]:
## Read data
train_df = gpd.read_file(f'{BASE_PATH}/data/1000samples.geojson', index_col=0)
# test_df = gpd.read_file(f'{BASE_PATH}/data/test.geojson', index_col=0)

#### Treatments

In [91]:
## Simple Treatments
train_df[DATE_COLUMNS] = train_df[DATE_COLUMNS].apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y', errors='coerce'))

#train_df['change_type'].map(CHANGE_TYPE_MAP)
train_df[CHANGE_STATUS_COLUMNS] = train_df[CHANGE_STATUS_COLUMNS].replace(CHANGE_STATUS_MAP)


/tmp/ipykernel_111183/4258756460.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df[CHANGE_STATUS_COLUMNS] = train_df[CHANGE_STATUS_COLUMNS].replace(CHANGE_STATUS_MAP)


In [92]:
model = LinearRegression()
num_samples = train_df.shape[0]
coef = np.zeros((num_samples))
ctt = 1e9*60*90*24
for i in range(num_samples):
    ctt
    y = np.array(train_df[CHANGE_STATUS_COLUMNS].iloc[i])
    x = np.array(train_df[DATE_COLUMNS].iloc[i].astype(int))[:,np.newaxis]/ctt

    y_nan_mask = np.isnan(y)
    if y_nan_mask.all():
        continue
    
    model.fit(x[~y_nan_mask],y[~y_nan_mask])
    coef[i] = model.coef_
    #print(y, train_df["change_type"].iloc[i])
train_df["civilizating_rate"] = coef



In [93]:
#print(train_df.head())